# Plot groups' analysis
## Set parameters

In [ ]:
# ####################################### GENERAL OPTIONS #######################################
branches_to_exclude = ["retina", "VS", "grv", "fiber tracts", "CB"]
#experiment_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/experiment/"
experiment_root = "./data/experiments/soumnya/experiment/"
#proof_root = "/run/user/1000/gvfs/smb-share:server=ich.techosp.it,share=ricerca/Lab Matteoli/Silva/collaborations/Mathias/soumnya/data/proof/"
proof_root = "./data/experiments/soumnya/proof/"


# ###################################### DATA LOAD OPTIONS ######################################
area_key = "Area um^2"
tracer_key = "Num AF647"
marker = "CFos"

# ###################################### DATA SAVE OPTIONS ######################################
animal_brain_mode = "sum"                                       # available options are: 'sum', 'mean'/'avg', 'std', 'variation'/'cvar'
save_animals = True
save_groups = True

# ###############################################################################################
from collections import namedtuple
GroupDirectory = namedtuple("GroupDirectory", "name dirs")

## Script's code
run all cell below

In [ ]:
import BraiAn
import os
from typing import List

In [ ]:
proof_input_path = os.path.join(proof_root, "QuPath_output")
experiment_input_path = os.path.join(experiment_root, "QuPath_output")

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = "./data/AllenMouseBrainOntology.json"
AllenBrain = BraiAn.AllenBrainHierarchy(path_to_allen_json, branches_to_exclude)
AllenBrain.select_from_csv("./data/AllenSummaryStructures.csv")
selected_regions = AllenBrain.get_selected_regions()
print(f"You selected {len(selected_regions)} Summary Structure regions.")

In [ ]:
proof_animal_dirs = [d for d in os.listdir(proof_input_path) if os.path.isdir(os.path.join(proof_input_path, d)) and not d.startswith(".")]
proof_slices = [BraiAn.SlicedBrain(animal_dir,
                                    os.path.join(proof_input_path, animal_dir),
                                    AllenBrain,
                                    area_key,
                                    tracer_key,
                                    marker,
                                    area_units="µm2")
                    for animal_dir in proof_animal_dirs]
print(f"Imported all brain slices from {str(len(proof_animal_dirs))} animals of 'proof'.")

experiment_animal_dirs = [d for d in os.listdir(experiment_input_path) if os.path.isdir(os.path.join(experiment_input_path, d)) and not d.startswith(".")]
experiment_slices = [BraiAn.SlicedBrain(animal_dir,
                                    os.path.join(experiment_input_path, animal_dir),
                                    AllenBrain,
                                    area_key,
                                    tracer_key,
                                    marker,
                                    area_units="µm2")
                    for animal_dir in experiment_animal_dirs]
print(f"Imported all brain slices from {str(len(experiment_animal_dirs))} animals of 'experiment'.")

In [ ]:
missing_in_experiment = [animal for animal in proof_animal_dirs if animal not in experiment_animal_dirs]
missing_in_proof = [animal for animal in experiment_animal_dirs if animal not in proof_animal_dirs]
print(f"Animals missing in 'experiment': {', '.join(missing_in_experiment)}")
print(f"Animals missing in 'proof': {', '.join(missing_in_proof)}")

In [ ]:
# NOTE: brains are being written WITH Left/Right discrimination
# If you desire to save them without, call AnimalBrain with hemisphere_distinction=False

min_slices = 3
proof_sum_brains: List[BraiAn.AnimalBrain] = [BraiAn.AnimalBrain(sliced_brain, mode=animal_brain_mode, min_slices=min_slices, hemisphere_distinction=False) for sliced_brain in proof_slices]
experiment_sum_brains: List[BraiAn.AnimalBrain] = [BraiAn.AnimalBrain(sliced_brain, mode=animal_brain_mode, min_slices=min_slices, hemisphere_distinction=False) for sliced_brain in experiment_slices]
#if save_animals:
#    for i in range(len(groups)):
#        for animal in groups_sum_brains[i]:
#            animal.write_all_brains(data_output_path)

In [ ]:
proof_selected_regions = BraiAn.AnimalBrain.filter_selected_regions(proof_sum_brains[0], AllenBrain).data
experiment_selected_regions = BraiAn.AnimalBrain.filter_selected_regions(experiment_sum_brains[0], AllenBrain).data
regions_in_both = list(set(experiment_selected_regions.index).intersection(set(proof_selected_regions.index)))

In [ ]:
proof_density = (proof_selected_regions[marker] / proof_selected_regions.area)[regions_in_both].mean()
experiment_density = (experiment_selected_regions[marker] / experiment_selected_regions.area)[regions_in_both].mean()

rebase_coefficient = proof_density / experiment_density
rebase_coefficient, rebase_coefficient * experiment_density, proof_density

In [ ]:
((experiment_selected_regions[marker] * rebase_coefficient) / experiment_selected_regions.area)[regions_in_both].mean()

In [ ]:
# We have to think very well what to do with rebase_coefficient. Can we use the rebase coefficient, computed on density, on marker numbers (surely not on area)?
# I would say yes.
# rebased_sum_brains: List[BraiAn.AnimalBrain] = [BraiAn.AnimalBrain(sliced_brain, mode="sum", min_slices=0, hemisphere_distinction=True) for sliced_brain in experiment_slices]

In [ ]:
import importlib
importlib.reload(BraiAn.sliced_brain)
importlib.reload(BraiAn.animal_brain)
importlib.reload(BraiAn)